In [ ]:
import numpy as np


In [ ]:
import dill
dill.settings['recurse'] = True
lamb=dill.load(open('lambda.pkl','rb'))

In [ ]:
import matplotlib.pyplot as plt
from sympy import latex

In [ ]:
def get_next_state(curr_q,curr_u,control,curr_t,dt,lamb):
    forces=control(curr_t)
    u_dot_f=lamb(curr_q,curr_u,*forces).flatten()
    next_q=curr_q+curr_u*dt
    next_u=curr_u+u_dot_f*dt
    return next_q,next_u


In [ ]:
def plot(t,y):
    plt.figure()
    plt.subplot(2,2,1)
    plt.plot(t, y[:, :3])
    plt.legend([latex(s, mode='inline') for s in 'xyz'])
    plt.xlabel('[sec]')
    plt.ylabel('[meters]')
    plt.title('a) Position vs Time')
    plt.subplot(2,2,2)
    plt.plot(t, np.rad2deg(y[:, 3:6]))
    plt.legend([latex(s, mode='inline') for s in 'ypr'])
    plt.xlabel('[sec]')
    plt.ylabel('[deg]')
    plt.title('b) Rotations vs Time')
    plt.subplot(2,2,3)
    plt.xlabel('[meters]')
    plt.ylabel('[meters]')
    plt.title('c) XY plane plot')
    plt.plot(y[:,0],y[:,1])
    plt.axis('equal')
    plt.subplot(2,2,4)
    plt.plot(t, np.rad2deg(y[:, 9:12]))
    plt.legend([latex(s, mode='inline') for s in ['Rz','Ry','Rx']])
    plt.xlabel('[sec]')
    plt.ylabel('[deg/sec]')
    plt.title('d) Angular velocities')
    

In [ ]:
def sim(controller):
    curr_q = np.zeros(6)
    curr_u = np.zeros(6)
    frames_per_sec = 60.0
    final_time = 40.0
    dt=1.0/frames_per_sec
    y=[]
    curr_time=0
    t = np.linspace(0.0, final_time, int(final_time * frames_per_sec))
    #lamb=get_next_state_lambda()
    for curr_time in t:
        #next_q,next_u=get_next_state(curr_q,curr_u,controller,curr_time,dt,subs)
        next_q,next_u=get_next_state(curr_q,curr_u,controller,curr_time,dt,lamb)
        next_q,next_u=next_q.flatten(),next_u.flatten()
        y.append(np.hstack((next_q,next_u)))
        curr_q,curr_u=next_q,next_u
        #print(next_u)
    y=np.array(y,dtype='float32')
    return(t,y)

In [ ]:
# l ,r f ,b
def controller2(t):
    if t<10: #left
        return [0,0,0,0.0, -1,1,-1,1]
    elif t<20:
        return [0,0,0,0.0, 1,-1,1,-1]
    elif t<30:
        return [0,0,0,0.0, 1,1,1,1]
    else:
        return [0,0,0,0.0, -1,-1,-1,-1]
t,y=sim(controller2)
plot(t,y)